In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
import PIL
import glob

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


In [1]:
# coding:utf-8
# import keras
# from keras.utils import np_utils
# from keras.layers.convolutional import Conv2D, MaxPooling2D
# from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation, Flatten
# from keras.preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np
import pandas as pdl
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
import os
from PIL import Image
import glob
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


imgwide = 400
imghigh = 400
#imgdata_dir = "/Users/sota/Labratory_Local/FoodRecognition/data/imgs"#in local enviolenment, so must change
imgdata_dir = "/home/stsubokura/FoodRecognition/data/"

imgs = []
label = []
    
folder = ["cucumberinside_imgs","cucumberoutside_imgs"]
image_size = 400
 
for index, name in enumerate(folder):
    dir = imgdata_dir + name
    files = glob.glob(dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("L")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        data = data.reshape(image_size, image_size, 1)
        imgs.append(data)
        label.append(index)

imgs = np.asarray(imgs)
print(imgs.shape)
imgs = imgs.astype('float32')
imgs = imgs / 255.0

label = np.asarray(label)
label = tf.keras.utils.to_categorical(label, 7)

# generate train and test data automatically, using scikit-learn
# how diside random state size???
img_train, img_test, label_train, label_test = train_test_split(imgs, label, test_size=0.25, random_state=111)
print(img_train.shape)


(4550, 400, 400, 1)
(3412, 400, 400, 1)


In [ ]:
epochs = 20
with tf.distribute.MirroredStrategy().scope():
    model = Sequential([
        Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=img_train.shape[1:]),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(32, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(7, activation='sigmoid' , kernel_regularizer = tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001) )
    ])

model.summary()

# コンパイル
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 実行。
history = model.fit(img_train, label_train, batch_size=256, epochs=epochs, validation_data=(img_test, label_test) )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [ ]:
plt.title('CNN ')
plt.plot(range(1, epochs+1), history.history['accuracy'], label="training")
plt.plot(range(1, epochs+1), history.history['val_accuracy'], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()